## Model Data using Cassandra


### Please just submit this notebook in the Submission and make sure each cell has been executed and output is clearly displayed.

### The aim of the project is to solve the three queries given below.

### Introduction

There is a music streaming app called SoundCloud, that has been using their music streaming app and collecting data on songs and user activity and their aim is to analyze this data especially understanding what songs users are listening to. Currently, they are not making use of a NoSQL db and they have the data stored as a CSV file, thus its difficult for them to query the data. So our task is to create a NoSQL database for helping them with the analysis.

#### Import Packages 

In [8]:
import pandas as pd
import numpy as np
import cassandra
import csv



##  The image below is a screenshot of what the data appears like in the event_data.csv

<img src="event_data_image.jpg">

#### Creating a Cluster

In [36]:
# Task: Make a connection to the cassandra instance on your local machine(127.0.0.1) and 
# Make a connection to the Cassandra instance on your local machine (127.0.0.1)
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider


auth_token = 'AstraCS:zjofYRUCLgJkmEKNCkziNJJD:7eaacd01c4625a783b87438904ae74ef84c08d70caf1a01922ce0c728ef0f15a'

# Configure connection
auth_provider = PlainTextAuthProvider(username='token', password=auth_token)
cluster = Cluster(auth_provider=auth_provider)

# Create a session
session = cluster.connect()

#### Create & Set Keyspace

In [37]:
# Task: Create a Keyspace and Set KEYSPACE to the keyspace specified above

session.execute("""
CREATE KEYSPACE IF NOT EXISTS soundcloud
WITH REPLICATION = 
{ 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)


# Setting keyspace
session.set_keyspace('soundcloud')


## List of Queries 

### 1. Find the artist_name, song_title and length of song the SoundCloud app history that was heard during  session_number = 338, and item_in_session_number  = 4


### 2. Find the artist_name, song_title (sorted by item_in_session_number) and name(fname and lname) of the user for user_id = 10, session_number = 182
    

### 3. Find every name(fname and lname) of the user from the SoundCloud app history that listened to the song_title 'All Hands Against His Own'




### Query1 Table1: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

In [38]:
## Task: Query 1: Find the artist_name, song_title and length of song the SoundCloud app history 
## that was heard during session_number = 338, and item_in_session_number = 4
## make use of create table command   

session.execute("""
CREATE TABLE IF NOT EXISTS song_in_session (
    session_number int,
    item_in_session_number int,
    artist_name text,
    song_title text,
    length float,
    PRIMARY KEY (session_number, item_in_session_number))
""")



### Let's insert our data into of table

In [39]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'

with open(file_name, encoding='utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # Skip header
    for row in csv_reader:
        query = "INSERT INTO song_in_session (session_number, item_in_session_number, artist_name, song_title, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(row[8]), int(row[3]), row[0], row[9], float(row[5])))


### Validate our Data Model using a SELECT

In [40]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
query = "SELECT artist_name, song_title, length FROM song_in_session WHERE session_number = 338 AND item_in_session_number = 4"
rows = session.execute(query)
for row in rows:
    print(row.artist_name, row.song_title, row.length)


Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query2 Table2: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key 

In [41]:
## Task: Query 2: Find the artist_name, song_title (sorted by item_in_session_number) and 
## name(fname and lname) of the user for user_id = 10, session_number = 182 
## make use of create table command    

session.execute("""
CREATE TABLE IF NOT EXISTS user_in_session (
    user_id int,
    session_number int,
    item_in_session_number int,
    artist_name text,
    song_title text,
    user_fname text,
    user_lname text,
    PRIMARY KEY ((user_id, session_number), item_in_session_number))
""")



### Let's insert our data into of table

In [42]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'
with open(file_name, encoding='utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # Skip header
    for row in csv_reader:
        query = "INSERT INTO user_in_session (user_id, session_number, item_in_session_number, artist_name, song_title, user_fname, user_lname)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(row[10]), int(row[8]), int(row[3]), row[0], row[9], row[1], row[4]))



### Validate our Data Model using a SELECT

In [43]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
query = "SELECT artist_name, song_title, user_fname, user_lname FROM user_in_session WHERE user_id = 10 AND session_number = 182"
rows = session.execute(query)
for row in rows:
    print(row.artist_name, row.song_title, row.user_fname, row.user_lname)



Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query3 Table3: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

In [44]:
## Task: Query 3: Find every name(first and lastname) of the user from the SoundCloud app history that listened 
## to the song_title 'All Hands Against His Own'
## make use of create table command  


session.execute("""
CREATE TABLE IF NOT EXISTS user_by_song (
    song_title text,
    user_id int,
    user_fname text,
    user_lname text,
    PRIMARY KEY (song_title, user_id))
""")


### Let's insert our data into of table

In [45]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'
with open(file_name, encoding='utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # Skip header
    for row in csv_reader:
        query = "INSERT INTO user_by_song (song_title, user_id, user_fname, user_lname)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (row[9], int(row[10]), row[1], row[4]))


### Validate our Data Model using a SELECT

In [46]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
query = "SELECT user_fname, user_lname FROM user_by_song WHERE song_title = 'All Hands Against His Own'"
rows = session.execute(query)
for row in rows:
    print(row.user_fname, row.user_lname)


Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [47]:
session.execute("DROP TABLE IF EXISTS song_in_session")

In [48]:
session.execute("DROP TABLE IF EXISTS user_in_session")

In [49]:
session.execute("DROP TABLE IF EXISTS user_by_song")

### Close the session and cluster connection¶

In [50]:
session.shutdown()

In [51]:
cluster.shutdown()